In [2]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Sat Dec  5 03:47:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    24W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [4]:
!pip install git+https://github.com/kernelmachine/allennlp.git@4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
!pip install transformers==2.4.1
!pip install pytorch-transformers==1.2.0
!git clone https://github.com/allenai/dont-stop-pretraining

!pip install adapter-transformers

  Cloning https://github.com/kernelmachine/allennlp.git (to revision 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7) to /tmp/pip-req-build-rcpchwio
  Running command git clone -q https://github.com/kernelmachine/allennlp.git /tmp/pip-req-build-rcpchwio
  Running command git checkout -q 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 30.9MB 179kB/s 
     |████████████████████████████████| 133kB 51.8MB/s 
     |████████████████████████████████| 133kB 51.3MB/s 
     |████████████████████████████████| 245kB 51.5MB/s 
     |████████████████████████████████| 317kB 50.3MB/s 
     |████████████████████████████████| 163kB 48.0MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 5.3MB 42.9MB/s 
     |████████████████████████████████| 71kB 9.5MB/s

In [5]:
!ls

dont-stop-pretraining  gdrive  sample_data


In [6]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [6]:
!ls gdrive/MyDrive/pre-trained

ag	  citation_intent  hyperpartisan_news  rct-20k
chemprot  helpfulness	   imdb		       scierc


In [7]:
%cd temp/adapticons/

/content/temp/adapticons


In [8]:
!ls

modeling  new_train_requirements.txt  README.md


In [8]:
%mkdir datasets
%cd datasets
%mkdir rct-20k
%cd rct-20k
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/rct-20k/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/rct-20k/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/rct-20k/test.jsonl
%cd ../..

/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/rct-20k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 34.7M  100 34.7M    0     0  9379k      0  0:00:03  0:00:03 --:--:-- 9381k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5959k  100 5959k    0     0  2809k      0  0:00:02  0:00:02 --:--:-- 2809k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5920k  100 5920k    0     0  2910k      0  0:00:02  0:00:02 --:--:-- 2910k
/content/temp/adapticons


In [10]:
!ls datasets/rct-20k

dev.jsonl  test.jsonl  train.jsonl


In [11]:
%cd modeling

/content/temp/adapticons/modeling


In [12]:
!ls ../../../gdrive/MyDrive/pre-trained

ag	  citation_intent  hyperpartisan_news  rct-20k
chemprot  helpfulness	   imdb		       scierc


In [17]:
!ls ../datasets

rct-20k


In [13]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/rct-20k/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 1 \
  --learning_rate 1e-4 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/RCT_Fusion_chem_rct \
  --task_name adapter_fusion \
  --do_predict \
  --load_best_model_at_end \
  --train_fusion \
  --model_name_or_path roberta-base \
  --adapter_config pfeiffer \
  --metric micro \
  --fusion_adapter_path1 ../../../gdrive/MyDrive/pre-trained/chemprot \
  --fusion_adapter_path2 ../../../gdrive/MyDrive/pre-trained/rct-20k \
  --overwrite_output_dir 

2020-12-04 03:20:27.865984: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/04/2020 03:20:29 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/04/2020 03:20:29 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/RCT_Fusion_chem_rct', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, lo

In [8]:
%mkdir datasets
%cd datasets
%mkdir citation
%cd citation
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl
%cd ../..

/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/citation
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0   289k      0  0:00:01  0:00:01 --:--:--  289k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30138  100 30138    0     0  35498      0 --:--:-- --:--:-- --:--:-- 35456
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38637  100 38637    0     0  46216      0 --:--:-- --:--:-- --:--:-- 46161
/content/temp/adapticons


In [11]:
%cd modeling

/content/temp/adapticons/modeling


In [13]:
!ls ../datasets/citation

dev.jsonl  test.jsonl  train.jsonl


In [14]:
!ls ../../../gdrive/MyDrive/downsample_sciee/

_25_pct  _25_pct_adapter  _50_pct  _50_pct_adapter  _75_pct  _75_pct_adapter


In [15]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_25_adapter \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_25_pct_adapter/ \
  --adapter_config pfeiffer \
  --metric macro \
  --overwrite_output_dir 

2020-12-05 03:57:57.981101: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 03:57:59 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 03:57:59 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_25_adapter', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup

In [16]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_50_adapter \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_50_pct_adapter/ \
  --adapter_config pfeiffer \
  --metric macro \
  --overwrite_output_dir 

2020-12-05 04:06:43.668070: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 04:06:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 04:06:45 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_50_adapter', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup

In [17]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_75_adapter \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_75_pct_adapter/ \
  --adapter_config pfeiffer \
  --metric macro \
  --overwrite_output_dir 

2020-12-05 04:13:52.468795: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 04:13:54 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 04:13:54 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_75_adapter', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup

In [18]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 1e-4 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_25_adapter_2 \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_25_pct_adapter/ \
  --adapter_config pfeiffer \
  --metric macro \
  --overwrite_output_dir 

2020-12-05 04:25:16.596318: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 04:25:18 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 04:25:18 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_25_adapter_2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, war

In [19]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 1e-4 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_50_adapter_2 \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_50_pct_adapter/ \
  --adapter_config pfeiffer \
  --metric macro \
  --overwrite_output_dir 

2020-12-05 04:32:17.212872: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 04:32:19 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 04:32:19 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_50_adapter_2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, war

In [20]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 1e-4 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_75_adapter_2 \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --train_adapter \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_75_pct_adapter/ \
  --adapter_config pfeiffer \
  --metric macro \
  --overwrite_output_dir 

2020-12-05 04:39:18.040322: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 04:39:19 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 04:39:19 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_75_adapter_2', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, war